In [ ]:
library("tidyverse")
library("knitr")
library("pander")
library("car")
library("reshape2")
library("ggplot2")
library('gridExtra')
library("plyr")
#library("nlme")
#library("multcomp")
library('magrittr')


In [ ]:
#One-way ANOVA health measures~group
df <- read_csv('~/Google Drive/WIN_Matter/analysis/data/health_demographics_subset.csv')

df <- mutate(data, Group=as.factor(Group), 
               Lab_ID=as.factor(Lab_ID))


difference_scores=data.frame(
Group=df$Group,
Lab_ID=df$Lab_ID,
Weight=df$wtkg_followup-df$wtkg_baseline,
BMI=df$BMI_followup-df$BMI_baseline,
w2H=df$waistIliac_followup-df$waistIliac_baseline,
DXA_Tissue=df$DXA_Tissue_pcfat_followup-df$DXA_Tissue_pcfat_baseline,
DXA_Region=df$DXA_Region_pcfat_followup-df$DXA_Region_pcfat_baseline,
VO2LMin=df$VO2LMin_followup-df$VO2LMin_baseline,
VO2mlkgmin=df$VO2mlkgmin_followup-df$VO2mlkgmin_baseline
)

final_df=difference_scores[complete.cases(difference_scores),]

#Repeated measures anova VO2
anova_results <- aov(VO2LMin ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova VO2mlkgmin
anova_results <- aov(VO2mlkgmin ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova DXA tissue
anova_results <- aov(DXA_Tissue ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova DXA tissue
anova_results <- aov(DXA_Region ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova DXA tissue
anova_results <- aov(w2H ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova DXA tissue
anova_results <- aov(BMI ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

#Repeated measures anova DXA tissue
anova_results <- aov(Weight ~ Group + Error(factor(Lab_ID)),
                     data = final_df)

summary(anova_results)

In [ ]:
#Get coefiecients that are significant 
get_coef_values= function(fiber, df){
    df=mutate(df, Sex=as.factor(Sex))
    lm_reg=lm(df[[fiber]]~PC2+PC1+PC3+Sex+Age+deltaFD, data=df)
    #change depending on what you are extracting
    coef=lm_reg$coefficients[2]
    #change depending on what you are extracting
    p=summary(lm_reg)$coefficients[2,4]
    if(p<.05){
        psig=c(1)
    }else{
        psig=c(0)
    }
    Fiber_name=c(fiber)
    PC2_lesser=c(coef)
    row_info=data.frame(Fiber_name, PC2_lesser, psig)
    return(row_info)
}

#open the df you want to use
df=read_csv('/WIN_Matter/analysis/data/PC2_lesser_mask.csv')
#make a list to iterate over all fibers
subset_fibers <- df %>% dplyr:: select(starts_with("f"))
fiberList=colnames(subset_fibers)
#making an output dataframe for the coefficient values
coef_vector=NULL
for (fiber in fiberList){
    row_info=get_coef_values(fiber, df)
    #Fiber_name=c(fiber)
    #PC1_greater=c(coef)
    #row_info=data.frame(Fiber_name, PC1_greater)
    coef_vector=rbind(coef_vector, row_info)
#write.csv(coef_vector, "/WIN_Matter/analysis/data/PC2_lesser_coef.csv", row.names=FALSE)
    
}


In [ ]:
#One-way ANOVA sig QA~ group 
PC1_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_lesser_sigOnly_fibers_grubbs.csv')
PC1_less<- mutate(PC1_less, tx_code=factor(tx_code, 
                  levels=1:3, 
                  labels=c("DIET","MOD-PA","HIGH-PA")))

            

PC1_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_greater_sigOnly_fibers_grubbs.csv')
PC1_great<- mutate(PC1_great, tx_code=factor(tx_code, 
                  levels=1:3, 
                  labels=c("DIET","MOD-PA","HIGH-PA")))

PC2_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_greater_sigOnly_fibers_grubbs.csv')
PC2_great<- mutate(PC2_great, tx_code=factor(tx_code, 
                  levels=1:3, 
                  labels=c("DIET","MOD-PA","HIGH-PA")))

PC2_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_lesser_sigOnly_fibers_grubbs.csv')
PC2_less<- mutate(PC2_less, tx_code=factor(tx_code, 
                  levels=1:3, 
                  labels=c("DIET","MOD-PA","HIGH-PA")))

#PC1 lesser anova 
PC1_less_aov<-aov(fiberMean~tx_code, data=PC1_less)
#summary(PC1_less.aov)
#pander(PC1_less_aov, digits=3, format="pandoc", caption="Anova Mean Fiber by Exercise vs Diet PC1(-)")

TukeyHSD(PC1_less_aov)

# Create Boxplot
boxplot(fiberMean ~ tx_code, data = PC1_less,
        xlab = "Group", ylab = expression(paste(Delta,'MeanFiber'['post-pre'])),
        main = "Mean Fiber by Group PC1(-)")
#dev.copy(png, '~/Google Drive/WIN_Matter/analysis/output/images/pc1_less_tx_code_collapsed.jpg')
dev.off()
# Calculate Descriptive Statistics
summary_PC1less_group <- PC1_less %>%
                                    group_by(tx_code) %>%
                                    dplyr::summarize(Min  = min(fiberMean, na.rm = TRUE),
                                                     Q1 = quantile(fiberMean, na.rm=TRUE)[2],
                                                     Median = quantile(fiberMean, na.rm=TRUE)[3],
                                                     Mean = mean(fiberMean, na.rm = TRUE),
                                                     Q3 = quantile(fiberMean, na.rm=TRUE)[4],
                                                     Max = max(fiberMean, na.rm = TRUE),
                                                     SD = sd(fiberMean, na.rm = TRUE),
                                                     N = n()
                                                     )

kable(summary_PC1less_group)

#PC1 greater anova 
PC1_great.aov<-aov(fiberMean~tx_code, data=PC1_great)
#summary(PC1_great.aov)
pander(PC1_great.aov, digits=3, format="pandoc", caption="Anova Mean Fiber by Exercise vs Diet PC1(+)")

TukeyHSD(PC1_great.aov)

# Create Boxplot
boxplot(fiberMean ~ tx_code, data = PC1_great,
        xlab = "Group", ylab = expression(paste(Delta,'MeanFiber'['post-pre'])),
        main = "Mean Fiber by Group PC1(+)")
dev.copy(png, '~/Google Drive/WIN_Matter/analysis/output/images/pc1_great_tx_code_collapsed.jpg')
dev.off()
# Calculate Descriptive Statistics
summary_PC1great_group <- PC1_great %>%
                                    group_by(tx_code) %>%
                                    dplyr::summarize(Min  = min(fiberMean, na.rm = TRUE),
                                                     Q1 = quantile(fiberMean, na.rm=TRUE)[2],
                                                     Median = quantile(fiberMean, na.rm=TRUE)[3],
                                                     Mean = mean(fiberMean, na.rm = TRUE),
                                                     Q3 = quantile(fiberMean, na.rm=TRUE)[4],
                                                     Max = max(fiberMean, na.rm = TRUE),
                                                     SD = sd(fiberMean, na.rm = TRUE),
                                                     N = n()
                                                     )

kable(summary_PC1great_group)

#PC2 lesser anova 
PC2_less.aov<-aov(fiberMean~tx_code, data=PC2_less)
#summary(PC2_less.aov)
pander(PC2_less.aov, digits=3, format="pandoc", caption="Anova Mean Fiber by Exercise vs Diet PC2(-)")

TukeyHSD(PC2_less.aov)

# Create Boxplot
boxplot(fiberMean ~ tx_code, data = PC2_less,
        xlab = "Group", ylab = expression(paste(Delta,'MeanFiber'['post-pre'])),
        main = "Mean Fiber by Group PC2(-)")
#dev.copy(png, '~/Google Drive/WIN_Matter/analysis/output/images/pc2_less_tx_code_collapsed.jpg')
dev.off()
# Calculate Descriptive Statistics
summary_PC2less_group <- PC2_less %>%
                                    group_by(tx_code) %>%
                                    dplyr::summarize(Min  = min(fiberMean, na.rm = TRUE),
                                                     Q1 = quantile(fiberMean, na.rm=TRUE)[2],
                                                     Median = quantile(fiberMean, na.rm=TRUE)[3],
                                                     Mean = mean(fiberMean, na.rm = TRUE),
                                                     Q3 = quantile(fiberMean, na.rm=TRUE)[4],
                                                     Max = max(fiberMean, na.rm = TRUE),
                                                     SD = sd(fiberMean, na.rm = TRUE),
                                                     N = n()
                                                     )

kable(summary_PC2less_group)

#PC2 greater anova 
PC2_great.aov<-aov(fiberMean~tx_code, data=PC2_great)
#summary(PC2_great.aov)
pander(PC2_great.aov, digits=3, format="pandoc", caption="Anova Mean Fiber by Exercise vs Diet PC2(+)")

TukeyHSD(PC2_great.aov)

# Create Boxplot
boxplot(fiberMean ~ tx_code, data = PC2_great,
        xlab = "Group", ylab = expression(paste(Delta,'MeanFiber'['post-pre'])),
        main = "Mean Fiber by Group PC2(+)")
#dev.copy(png, '~/Google Drive/WIN_Matter/analysis/output/images/pc2_great_tx_code_collapsed.jpg')
dev.off()
# Calculate Descriptive Statistics
summary_PC2great_group <- PC2_great %>%
                                    group_by(tx_code) %>%
                                    dplyr::summarize(Min  = min(fiberMean, na.rm = TRUE),
                                                     Q1 = quantile(fiberMean, na.rm=TRUE)[2],
                                                     Median = quantile(fiberMean, na.rm=TRUE)[3],
                                                     Mean = mean(fiberMean, na.rm = TRUE),
                                                     Q3 = quantile(fiberMean, na.rm=TRUE)[4],
                                                     Max = max(fiberMean, na.rm = TRUE),
                                                     SD = sd(fiberMean, na.rm = TRUE),
                                                     N = n()
                                                     )

kable(summary_PC2great_group)



In [ ]:
#Multiple regression analysis cognitive measures and significant fibers
get_coef_values= function(fiber, df, pc){
    df=mutate(df, Sex=as.factor(Sex))
    #IGTp 
    IGTp_lm=lm(IGT_p_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    IGTp_SE=format(round(summary(IGTp_lm)$coefficients[2,2], digits=2))
    IGTp_pval=summary(IGTp_lm)$coefficients[2,4]
    if(IGTp_pval<.05){
        IGTp_coef=paste(format((IGTp_lm$coefficients[2]), digits=4),'*')
    }else{
        IGTp_coef=format((IGTp_lm$coefficients[2]), digits=4)
    }
    #IGTq
    IGTq_lm=lm(IGT_q_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    IGTq_SE=format(round(summary(IGTq_lm)$coefficients[2,2], digits=2))
    IGTq_pval=summary(IGTq_lm)$coefficients[2,4]
    if(IGTq_pval<.05){
        IGTq_coef=paste(format((IGTq_lm$coefficients[2]), digits=4),'*')
    }else{
        IGTq_coef=format((IGTq_lm$coefficients[2]),digits=4)
    }
    #TS RT
    TS_RT_lm=lm(TS_RT_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    TS_RT_SE=format(round(summary(TS_RT_lm)$coefficients[2,2], digits=2))
    TS_RT_pval=summary(TS_RT_lm)$coefficients[2,4]
    if(TS_RT_pval<.05){
        TS_RT_coef=paste(format((TS_RT_lm$coefficients[2]), digits=4),'*')
    }else{
        TS_RT_coef=format((TS_RT_lm$coefficients[2]), digits=4)
    }
    #TS ACC
    TS_ACC_lm=lm(TS_ACC_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    TS_ACC_SE=format(round(summary(TS_ACC_lm)$coefficients[2,2], digits=2))
    TS_ACC_pval=summary(TS_ACC_lm)$coefficients[2,4]
    if(TS_ACC_pval<.05){
        TS_ACC_coef=paste(format((TS_ACC_lm$coefficients[2]), digits=1),'*')
    }else{
        TS_ACC_coef=format((TS_ACC_lm$coefficients[2]), digits=1)
    }
    #nback RT
    nback_RT_lm=lm(nback_RT_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    nback_RT_SE=format(round(summary(nback_RT_lm)$coefficients[2,2], digits=2))
    nback_RT_pval=summary(nback_RT_lm)$coefficients[2,4]
    if(nback_RT_pval<.05){
        nback_RT_coef=paste(format((nback_RT_lm$coefficients[2]), digits=4),'*')
    }else{
        nback_RT_coef=format((nback_RT_lm$coefficients[2]), digits=4)
    }
    #nback ACC
    nback_ACC_lm=lm(nback_ACC_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    nback_ACC_SE=format(round(summary(nback_ACC_lm)$coefficients[2,2], digits=2))
    nback_ACC_pval=summary(nback_ACC_lm)$coefficients[2,4]
    if(nback_ACC_pval<.05){
        nback_ACC_coef=paste(format((nback_ACC_lm$coefficients[2]), digits=1),'*')
    }else{
        nback_ACC_coef=format((nback_ACC_lm$coefficients[2]),digits=1)
    }
    #stroop int
    stroop_int_lm=lm(stroop_int_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    stroop_int_SE=format(round(summary(stroop_int_lm)$coefficients[2,2], digits=2))
    stroop_int_pval=summary(stroop_int_lm)$coefficients[2,4]
    if(stroop_int_pval<.05){
        stroop_int_coef=paste(format((stroop_int_lm$coefficients[2]), digits=4),'*')
    }else{
        stroop_int_coef=format((stroop_int_lm$coefficients[2]), digits=4)
    }
    #stroop fac
    stroop_fac_lm=lm(stroop_fac_diff~df[[fiber]]+Sex+Age+deltaFD, data=df)
    stroop_fac_SE=format(round(summary(stroop_fac_lm)$coefficients[2,2], digits=2))
    stroop_fac_pval=summary(stroop_fac_lm)$coefficients[2,4]
    if(stroop_fac_pval<.05){
        stroop_fac_coef=paste(format((stroop_fac_lm$coefficients[2]), digits=4),'*')
    }else{
        stroop_fac_coef=format((stroop_fac_lm$coefficients[2]), digits=4)
    }
    #extracting all relevant information from each cogfactor analysis and aggregating to df
    PC=c(pc)
    IGT_p_diff=c(IGTp_coef)
    IGT_q_diff=c(IGTq_coef)
    TS_RT_diff=c(TS_RT_coef)
    TS_ACC_diff=c(TS_ACC_coef)
    stroop_int_diff=c(stroop_int_coef)
    stroop_fac_diff=c(stroop_fac_coef)
    nback_RT_diff=c(nback_RT_coef)
    nback_ACC_diff=c(nback_ACC_coef)
    row_info=data.frame(PC, IGT_p_diff, IGTp_SE, IGT_q_diff, IGTq_SE, TS_RT_diff, TS_RT_SE, TS_ACC_diff, TS_ACC_SE, stroop_int_diff, stroop_int_SE, stroop_fac_diff, stroop_fac_SE, nback_RT_diff, nback_RT_SE, nback_ACC_diff, nback_ACC_SE) 
    return(row_info)
}

#open the df you want to use
PC1_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_greater_sigOnly_fibers_grubbs.csv')
PC1_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_lesser_sigOnly_fibers_grubbs.csv')
PC2_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_greater_sigOnly_fibers_grubbs.csv')
PC2_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_lesser_sigOnly_fibers_grubbs.csv')


PC1_great_coef=get_coef_values('fiberMean', PC1_great, 'PC1(+)')
PC1_less_coef=get_coef_values('fiberMean', PC1_less, 'PC1(-)')
PC2_great_coef=get_coef_values('fiberMean', PC2_great, 'PC2(+)')
PC2_less_coef=get_coef_values('fiberMean', PC2_less, 'PC2(-)')
coef_vector=rbind(PC1_great_coef, PC1_less_coef, PC2_great_coef, PC2_less_coef)
#write.csv(coef_vector, "/data/dataDB/WIN_Matter/analysis/data/cogFactors_fibers_coef.csv", row.names=FALSE)
#rename columns to make sense of table
names(coef_vector)=c("PC and fiber direction", "IGT p coef", "IGT p SE", "IGT q coef", 
                     "IGT q SE", "Task Switch RT coef", "Task Switch RT SE", "Task Switch ACC coef",
                     "Task Switch ACC SE", "Stroop Interference coef", 
                     "Stroop Interference SE","Stroop Facilitation coef", 
                     "Stroop Faciliation SE", "nback RT coef", 
                     "nback RT SE", "nback ACC coef", "nback ACC SE")
rownames(coef_vector)=NULL
#write.csv(coef_vector, "~/Google Drive/WIN_Matter/analysis/data/cogFactors_fibers_coef.csv", row.names=FALSE)
#png("~/Google Drive/WIN_Matter/analysis/output/results/cogFacors_fibers_coef_grubbs.png", height=300, width=1900)
#p<-tableGrob(coef_vector, rows=NULL)
#grid.arrange(p)
#dev.off()

IGTdata=coef_vector[c(1:5)]
#png("~/Google Drive/WIN_Matter/analysis/output/results/IGT_fibers_coef_grubbs.png", height=300, width=1900)
p<-tableGrob(IGTdata, rows=NULL)
grid.arrange(p)
dev.off()

TSdata=coef_vector[c(1, 6:9)]
#png("~/Google Drive/WIN_Matter/analysis/output/results/TS_fibers_coef_grubbs.png", height=300, width=1900)
p<-tableGrob(TSdata, rows=NULL)
grid.arrange(p)
dev.off()

stroop_data=coef_vector[c(1, 10:13)]
#png("~/Google Drive/WIN_Matter/analysis/output/results/stroop_fibers_coef_grubbs.png", height=300, width=1900)
p<-tableGrob(stroop_data, rows=NULL)
grid.arrange(p)
dev.off()

nbackdata=coef_vector[c(1, 14:17)]
#png("~/Google Drive/WIN_Matter/analysis/output/results/nback_fibers_coef_grubbs.png", height=300, width=1900)
p<-tableGrob(nbackdata, rows=NULL)
grid.arrange(p)
dev.off()

nbackIGT=coef_vector[c(1:5, 14:17)]
#png("~/Google Drive/WIN_Matter/analysis/output/results/nbackIGT_fibers_coef_grubbs.png", height=300, width=1900)
p<-tableGrob(nbackIGT, rows=NULL)
grid.arrange(p)
dev.off()


In [3]:
#Bayes Factor 
PC1_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_lesser_sigOnly_fibers_grubbs.csv')
#PC1_less<- mutate(PC1_less, tx_code=factor(tx_code, 
#                  levels=1:3, 
#                  labels=c("DIET","MOD-PA","HIGH-PA")))

PC1_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC1_greater_sigOnly_fibers_grubbs.csv')
#PC1_great<- mutate(PC1_great, tx_code=factor(tx_code, 
#                  levels=1:3, 
#                  labels=c("DIET","MOD-PA","HIGH-PA")))

PC2_great<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_greater_sigOnly_fibers_grubbs.csv')
#PC2_great<- mutate(PC2_great, tx_code=factor(tx_code, 
#                  levels=1:3, 
#                  labels=c("DIET","MOD-PA","HIGH-PA")))

PC2_less<- read_csv('~/Google Drive/WIN_Matter/analysis/data/PC2_lesser_sigOnly_fibers_grubbs.csv')
#PC2_less<- mutate(PC2_less, tx_code=factor(tx_code, 
#                  levels=1:3, 
#                  labels=c("DIET","MOD-PA","HIGH-PA")))


#Bayes table 
model.BF  = matrix(data=NA, nrow=4, ncol=1)

BF.PC1_great.full = BIC(lm(fiberMean~tx_code, data=PC1_great))
BF.PC1_great.null = BIC(lm(fiberMean~1, data=PC1_great))
BF.PC1_great.BayesFactor = exp((BF.PC1_great.full -BF.PC1_great.null)/2) 
model.BF[1,1]=BF.PC1_great.BayesFactor

BF.PC1_less.full = BIC(lm(fiberMean~tx_code, data=PC1_less))
BF.PC1_less.null = BIC(lm(fiberMean~1, data=PC1_less))
BF.PC1_less.BayesFactor = exp((BF.PC1_less.full -BF.PC1_less.null)/2) 
model.BF[2,1]=BF.PC1_less.BayesFactor

BF.PC2_great.full = BIC(lm(fiberMean~tx_code, data=PC2_great))
BF.PC2_great.null = BIC(lm(fiberMean~1, data=PC2_great))
BF.PC2_great.BayesFactor = exp((BF.PC2_great.full -BF.PC2_great.null)/2) 
model.BF[3,1]=BF.PC2_great.BayesFactor

BF.PC2_less.full = BIC(lm(fiberMean~tx_code, data=PC2_less))
BF.PC2_less.null = BIC(lm(fiberMean~1, data=PC2_less))
BF.PC2_less.BayesFactor = exp((BF.PC2_less.full -BF.PC2_less.null)/2) 
model.BF[4,1]=BF.PC2_less.BayesFactor

colnames(model.BF)=c('Group')
rownames(model.BF)=c('PC1 Greater','PC1 Lesser','PC2 Greater','PC2 Lesser')
model.BF

pander(model.BF)
#write.csv(model.BF, file='~/Google Drive/WIN_Matter/analysis/output/images/Bayes/BF.csv')
#png(file="~/Google Drive/WIN_Matter/analysis/output/images/Bayes/BF_lm.png") 
#writePNG(model.BF)


Attaching package: ‘car’

The following object is masked from ‘package:dplyr’:

    recode

The following object is masked from ‘package:purrr’:

    some

Parsed with column specification:
cols(
  sub_id = col_character(),
  tx_code = col_double(),
  Age = col_double(),
  Sex = col_double(),
  deltaFD = col_double(),
  IGT_p_diff = col_double(),
  IGT_q_diff = col_double(),
  TS_RT_diff = col_double(),
  TS_ACC_diff = col_double(),
  stroop_int_diff = col_double(),
  stroop_fac_diff = col_double(),
  nback_RT_diff = col_double(),
  nback_ACC_diff = col_double(),
  PC1 = col_double(),
  PC2 = col_double(),
  PC3 = col_double(),
  fiberMean = col_double()
)
Parsed with column specification:
cols(
  sub_id = col_character(),
  tx_code = col_double(),
  Age = col_double(),
  Sex = col_double(),
  deltaFD = col_double(),
  IGT_p_diff = col_double(),
  IGT_q_diff = col_double(),
  TS_RT_diff = col_double(),
  TS_ACC_diff = col_double(),
  stroop_int_diff = col_double(),
  stroop_fac_diff =

,Group
PC1 Greater,12.994915
PC1 Lesser,3.917758
PC2 Greater,3.056901
PC2 Lesser,8.627279




|     &nbsp;      | Group |
|:---------------:|:-----:|
| **PC1 Greater** | 12.99 |
| **PC1 Lesser**  | 3.918 |
| **PC2 Greater** | 3.057 |
| **PC2 Lesser**  | 8.627 |



In [2]:
#Table 2. post-pre mean and sd by intervnetion groups
df= read.csv('~/Google\ Drive/WIN_Matter/analysis/data/health_demographics_subset.csv')
df$Sex <- as.factor(df$Sex)


difference_scores=data.frame(
Group=df$Group,
Age=df$Age,
Sex=df$Sex,
FD=df$FDdelta,
Weight=df$wtkg_followup-df$wtkg_baseline,
BMI=df$BMI_followup-df$BMI_baseline,
waistIliac=df$waistIliac_followup-df$waistIliac_baseline,
DXA_Tissue=df$DXA_Tissue_pcfat_followup-df$DXA_Tissue_pcfat_baseline,
DXA_Region=df$DXA_Region_pcfat_followup-df$DXA_Region_pcfat_baseline,
VO2LMin=df$VO2LMin_followup-df$VO2LMin_baseline,
VO2mlkgmin=df$VO2mlkgmin_followup-df$VO2mlkgmin_baseline
)

final_df=difference_scores[complete.cases(difference_scores),]
sapply(final_df, sd, na.rm=TRUE)

aggregate(final_df[, 2:11], list(final_df$Group), mean)
aggregate(final_df[, 2:11], list(final_df$Group), sd)

#t-tests
diet_df=subset(final_df, Group=='DIET')
mod_pa_df=subset(final_df, Group=='MOD-PA')
high_pa_df=subset(final_df, Group=='HIGH-PA')

t.test(final_df$VO2LMin)
t.test(diet_df$VO2LMin)
t.test(mod_pa_df$VO2LMin)
t.test(high_pa_df$VO2LMin)

t.test(final_df$VO2mlkgmin)
t.test(diet_df$VO2mlkgmin)
t.test(mod_pa_df$VO2mlkgmin)
t.test(high_pa_df$VO2mlkgmin)


t.test(final_df$Weight)
t.test(diet_df$Weight)
t.test(mod_pa_df$Weight)
t.test(high_pa_df$Weight)

t.test(final_df$FD)
t.test(diet_df$FD)
t.test(mod_pa_df$FD)
t.test(high_pa_df$FD)

t.test(final_df$BMI)
t.test(diet_df$BMI)
t.test(mod_pa_df$BMI)
t.test(high_pa_df$BMI)

t.test(final_df$waistIliac)
t.test(diet_df$waistIliac)
t.test(mod_pa_df$waistIliac)
t.test(high_pa_df$waistIliac)

t.test(final_df$DXA_Tissue)
t.test(diet_df$DXA_Tissue)
t.test(mod_pa_df$DXA_Tissue)
t.test(high_pa_df$DXA_Tissue)

t.test(final_df$DXA_Region)
t.test(diet_df$DXA_Region)
t.test(mod_pa_df$DXA_Region)
t.test(high_pa_df$DXA_Region)

#write.csv(final_df, file='~/Google\ Drive/WIN_Matter/analysis/data/health_demographics_diffscores.csv')


Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
“Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector.”Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
“Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector.”

Group        Age        Sex         FD     Weight        BMI waistIliac 
0.77774289 8.88250534 0.39192085 1.94815730 7.80931162 2.62741255 8.08553178 
DXA_Tissue DXA_Region    VO2LMin VO2mlkgmin 
0.04417508 0.04346797 0.22298274 2.96040744

Warning message in mean.default(X[[i]], ...):
“argument is not numeric or logical: returning NA”Warning message in mean.default(X[[i]], ...):
“argument is not numeric or logical: returning NA”Warning message in mean.default(X[[i]], ...):
“argument is not numeric or logical: returning NA”

Group.1,Age,Sex,FD,Weight,BMI,waistIliac,DXA_Tissue,DXA_Region,VO2LMin,VO2mlkgmin
DIET,43.02632,NA,0.16808155,-9.947368,-3.500000,-8.210526,-0.05185789,-0.05141053,-0.114605263,1.628947
HIGH-PA,44.75758,NA,-0.03697742,-9.666667,-3.393939,-9.969697,-0.05593333,-0.05541212,-0.009787879,2.866667
MOD-PA,45.10000,NA,-0.81113105,-11.750000,-4.250000,-9.750000,-0.06072000,-0.06024500,-0.106700000,2.140000


Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
“Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector.”Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
“Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector.”Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
“Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector.”

...


	One Sample t-test

data:  high_pa_df$DXA_Tissue
t = -7.0874, df = 32, p-value = 4.864e-08
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.07200861 -0.03985806
sample estimates:
  mean of x 
-0.05593333 



	One Sample t-test

data:  final_df$DXA_Region
t = -12.027, df = 90, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.06385594 -0.04575065
sample estimates:
 mean of x 
-0.0548033 



	One Sample t-test

data:  diet_df$DXA_Region
t = -7.0346, df = 37, p-value = 2.549e-08
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.06621844 -0.03660261
sample estimates:
  mean of x 
-0.05141053 



	One Sample t-test

data:  mod_pa_df$DXA_Region
t = -6.7441, df = 19, p-value = 1.918e-06
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.07894203 -0.04154797
sample estimates:
mean of x 
-0.060245 



	One Sample t-test

data:  high_pa_df$DXA_Region
t = -7.1407, df = 32, p-value = 4.189e-08
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.07121891 -0.03960534
sample estimates:
  mean of x 
-0.05541212 
